# 5 simulate a scenario

change ${CORE_ROOT} to your path of core.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of uoa-poc2.

In [ ]:
export PJ_ROOT="${HOME}/uoa-poc2"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/uoa-poc2
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `uoa-poc2`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## setup alias

In [ ]:
alias now="python -c 'import datetime; print(datetime.datetime.now().strftime(\"%Y-%m-%dT%H:%M:%S.%f+09:00\"))'"
alias iso8601="python -c 'import datetime; print(datetime.datetime.now(tz=datetime.timezone.utc).isoformat(timespec=\"seconds\"))'"

## simulate a scenario

### 1. ship an item to a destination

#### post a shipment data (as ORDERING system)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  https://api.${DOMAIN}/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
  "destination": {
    "name": "会議室1"
  },
  "updated": [
    {
      "prev_quantity": 5,
      "new_quantity": 4,
      "reservation": 1,
      "title": "美味しい天然水（2L）",
      "place": "倉庫1"
    }
  ],
  "caller": "zaico-extensions"
}
__EOS__

example)

```
HTTP/1.1 201 Created
server: envoy
date: Fri, 15 Nov 2019 02:06:05 GMT
content-type: application/json
content-length: 160
access-control-allow-origin: *
x-envoy-upstream-service-time: 305

{"caller":"ordering","delivery_robot":{"id":"delivery_robot_02"},"order":{"destination":"place_Jdash","source":"place_V","via":["place_L"]},"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (157 bytes))
{"send_cmd":{"time":"2019-11-15T11:06:05.162+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.5,"y":-1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.46}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)
```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:06:05.162+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.5
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
---
```

#### confirm delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=send_cmd_status,current_routes,navigating_waypoints,remaining_waypoints_list" | jq .

confirm below:

* `send_cmd_status` is `PENDING`.
* `current_routes`, `navigating_waypoints` and `remaining_waypoints_list` is filled.
* `navigating_waypoints.waypoints` is the same of `waypoints` of MQTT message and ros message.

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* all token is not locked. (`is_locked` is `false`)
* all `lock_owner_id` are empty.
* all `waitings` are empty list.

### 2. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:00:00.000+09:00'
received_time: '2019-11-13T14:51:51.416+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: -0.5
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (283 bytes))
{"send_cmd": {"received_time": "2019-11-13T14:51:51.416+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}], "result": "ack", "time": "2019-11-08T14:00:00.000+09:00"}}
```

#### confirm delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=send_cmd_status,send_cmd_info" | jq .

confirm below:
* `send_cmd_status` turns to `OK` and `send_cmd_info` has the result message.

### 3. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T14:51:51.416+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (531 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.2, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:11:38.571021+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (96 bytes))
{"send_state":{"time":"2019-11-15T11:11:38.738+09:00","state":"moving","destination":"倉庫1"}}
```

#### confirm log messages

example)
```
2019/11/15 02:11:38 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode= , next_mode=navi
2019/11/15 02:11:38 [   INFO] src.api - publish new state to robot ui(robotui_02), current_state= , next_state=moving, destination=倉庫1
```

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=current_mode,current_state" | jq .

example)

`current_mode` is `navi`, and `current_state` is `moving`

```json
{
  "id": "delivery_robot_02",
  "type": "delivery_robot",
  "current_mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-12T17:35:14.048+09:00"
      }
    }
  },
  "current_state": {
    "type": "string",
    "value": "moving",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-12T17:35:14.059+09:00"
      }
    }
  }
}
```

#### send a `navi` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T14:51:52.416+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.2, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_02/attrs` is sent.
* no MQTT message to `/robot_ui/robotui_02/cmd` is sent.
* no log message is shown.

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=current_mode,current_state" | jq .

`current_mode` and `current_state` were not be changed because the`mode` sent from delivery_robot_02 was not changed.

### 4. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-08T14:01:00.000+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -1.49, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.45}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (537 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.45, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.49, "x": -0.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:13:29.364882+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (204 bytes))
{"send_cmd":{"time":"2019-11-15T11:13:29.537+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":4.71,"y":-1.99,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T11:13:29.580+09:00","token":"token_a","mode":"lock","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:13:29.537+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.71
      y: -1.99
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)
```
2019/11/15 02:13:29 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:13:29 [   INFO] src.token - lock token (token_a) by delivery_robot_02
2019/11/15 02:13:29 [   INFO] src.api - move robot(delivery_robot_02) to "place_L" (waypoints=[{'point': {'x': 0.5, 'y': -2, 'z': 0}, 'angle': None}, {'point': {'x': 4.71, 'y': -1.99, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 02:13:29 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_a, mode=lock, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is locked by `delivery_robot_02`, but no robot is waiting for this token yet.
* `token_b` and `token_c` is not locked.

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=current_mode,current_state" | jq .

example)

`current_mode` is changed to `standby`, but `current_state` is not changed.

```json
{
  "id": "delivery_robot_02",
  "type": "delivery_robot",
  "current_mode": {
    "type": "string",
    "value": "standby",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-13T14:57:20.575+09:00"
      }
    }
  },
  "current_state": {
    "type": "string",
    "value": "moving",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-13T14:55:12.979+09:00"
      }
    }
  }
}
```

#### send a `standby` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T14:55:12.979+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -1.49, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.45}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_02/attrs` is sent.
* no MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent.
* no log message is shown.

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=current_mode,current_state" | jq .

`current_mode` and `current_state` were not be changed because the`mode` sent from delivery_robot_02 was not changed.

### 5. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T14:55:13.979+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -1.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.4}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.4, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.1, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:16:44.344430+09:00"}
```

#### confirm log messages

example)
```
2019/11/15 02:16:44 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
```

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_02)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}/?attrs=current_mode,current_state" | jq .

`current_mode` is changed to `navi`, but `current_state` is not changed.

### 6. ship items sotred different warehouses each other to another destination

#### post a shipment data (as WAREHOUSE system)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-type: application/json"  https://api.${DOMAIN}/controller/api/v1/shipments/ -X POST -d @-<<__EOS__
{
  "destination": {
    "name": "会議室2"
  },
  "updated": [
    {
      "prev_quantity": 4,
      "new_quantity": 3,
      "reservation": 1,
      "title": "美味しい天然水（2L）",
      "place": "倉庫1"
    },
    {
      "prev_quantity": 10,
      "new_quantity": 8,
      "reservation": 2,
      "title": "地域名産炭酸水（500ml）",
      "place": "倉庫2"
    }
  ]
}
__EOS__

example)

```
HTTP/1.1 201 Created
server: envoy
date: Fri, 15 Nov 2019 02:18:38 GMT
content-type: application/json
content-length: 171
access-control-allow-origin: *
x-envoy-upstream-service-time: 354

{"caller":"warehouse","delivery_robot":{"id":"delivery_robot_01"},"order":{"destination":"place_Idash","source":"place_Q","via":["place_L","place_G"]},"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (157 bytes))
{"send_cmd":{"time":"2019-11-15T11:18:38.350+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.5,"y":-1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.46}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T11:18:38.350+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.5
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
---
```

#### confirm delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/?attrs=send_cmd_status,current_routes,navigating_waypoints,remaining_waypoints_list" | jq .

confirm below:

* `send_cmd_status` is `PENDING`.
* `current_routes`, `navigating_waypoints` and `remaining_waypoints_list` is filled.
* `navigating_waypoints.waypoints` is the same of `waypoints` of MQTT message and ros message.

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, but no robot is waiting for this token yet.
* `token_b` and `token_c` is not locked.

### 7. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:01:14.279+09:00'
received_time: '2019-11-13T15:01:14.179+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: -0.5
      y: -1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.46
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (283 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:01:14.179+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:01:14.279+09:00"}}
```

#### confirm delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/?attrs=send_cmd_status,send_cmd_info" | jq .

confirm below:
* `send_cmd_status` turns to `OK` and `send_cmd_info` has the result message.

### 8. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:01:15.179+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (531 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.2, "x": 0.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:21:51.073989+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (96 bytes))
{"send_state":{"time":"2019-11-15T11:21:51.237+09:00","state":"moving","destination":"倉庫1"}}
```

#### confirm log messages

example)
```
2019/11/15 02:21:51 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode= , next_mode=navi
2019/11/15 02:21:51 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state= , next_state=moving, destination=倉庫1
```

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/?attrs=current_mode,current_state" | jq .

example)

`current_mode` is `navi`, and `current_state` is `moving`

```json
{
  "id": "delivery_robot_01",
  "type": "delivery_robot",
  "current_mode": {
    "type": "string",
    "value": "navi",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-13T15:05:00.209+09:00"
      }
    }
  },
  "current_state": {
    "type": "string",
    "value": "moving",
    "metadata": {
      "TimeInstant": {
        "type": "datetime",
        "value": "2019-11-13T15:05:00.221+09:00"
      }
    }
  }
}
```

#### send a `navi` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:01:15.189+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.0, y: -0.2, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_01/attrs` is sent.
* no MQTT message to `/robot_ui/robotui_01/cmd` is sent.
* no log message is shown.

#### confirm the `current_mode` and `current_state` of delivery robot entity (delivery_robot_01)

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}/?attrs=current_mode,current_state" | jq .

`current_mode` and `current_state` were not be changed because the`mode` sent from delivery_robot_01 was not changed.

### 9. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:01:18.189+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -1.51, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (537 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.51, "x": -0.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.46, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:23:07.141363+09:00"}
```

#### confirm the MQTT and ROS message

* no MQTT message is sent to `/delivery_robot/delivery_robot_01/cmd`
* no ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent
* a `suspend` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically because `token_a` has been locked by `delivery_robot_02`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T11:23:07.305+09:00","token":"token_a","mode":"suspend","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

`token_a` has been locked by `delivery_robot_02`, so `delivery_robot_01` can not move into restlicted area

#### confirm log messages

example)
```
2019/11/15 02:23:07 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 02:23:07 [   INFO] src.token - wait token (token_a) by delivery_robot_01
2019/11/15 02:23:07 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=suspend, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is now waiting for this token.
* `token_b` and `token_c` is not locked.

#### send a `standby` ros message again

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:01:18.289+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -1.51, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
destination:
  point: {x: -0.5, y: -1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.46}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

confirm below:

* a MQTT message to `/delivery_robot/delivery_robot_01/attrs` is sent.
* no MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent.
* no log message is shown.

### 10. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:02:18.289+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -0.8, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.8}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.8, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:31:27.429845+09:00"}
```

### 11. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:04:18.289+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.71, y: -1.99, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 4.71, y: -1.99, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (538 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 4.71, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 4.71, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:31:50.660751+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (97 bytes))
{"send_state":{"time":"2019-11-15T11:31:50.796+09:00","state":"picking","destination":"倉庫1"}}
```

#### confirm log messages

example)
```
2019/11/15 02:31:50 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:31:50 [   INFO] src.api - publish new state to robot ui(robotui_02), current_state=moving, next_state=picking, destination=倉庫1
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is now waiting for this token.
* `token_b` and `token_c` is not locked.

### 12. emulate the completion of  picking operation ( `delivery_robot_02`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_02}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 15 Nov 2019 02:33:18 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 167

{"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (202 bytes))
{"send_cmd":{"time":"2019-11-15T11:33:18.229+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":-0.5,"y":-2.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.8}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:33:18.229+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -0.5
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_02`, and `delivery_robot_01` is still waiting for this token.
* `token_b` and `token_c` is not locked.

### 13. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:11:57.778+09:00'
received_time: '2019-11-13T15:11:56.778+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -0.5
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (341 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:11:56.778+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:11:57.778+09:00"}}
```

### 14. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:11:57.778+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.6, y: -1.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.01}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.9, "x": 0.6, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:34:59.123268+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (99 bytes))
{"send_state":{"time":"2019-11-15T11:34:59.258+09:00","state":"moving","destination":"会議室1"}}
```

#### confirm log messages

example)
```
2019/11/15 02:34:59 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
2019/11/15 02:34:59 [   INFO] src.api - publish new state to robot ui(robotui_02), current_state=picking, next_state=moving, destination=会議室1
```

### 15. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:14:19.249+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -2.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
destination:
  point: {x: -0.5, y: -2.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:35:49.879274+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (206 bytes))
{"send_cmd":{"time":"2019-11-15T11:35:50.052+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.94,"y":-5.9,"z":0},"angle":null},{"point":{"x":0.7,"y":-7.4,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.15}}]}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:35:50.052+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.94
      y: -5.9
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.7
      y: -7.4
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.15
---
```

#### confirm that a `release` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (167 bytes))
{"send_token_info":{"time":"2019-11-15T11:35:50.098+09:00","token":"token_a","mode":"release","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (204 bytes))
{"send_cmd":{"time":"2019-11-15T11:35:50.192+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":4.71,"y":-1.99,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T11:35:50.192+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 4.71
      y: -1.99
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm that `resume` and `lock` MQTT messages to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (166 bytes))
{"send_token_info":{"time":"2019-11-15T11:35:50.243+09:00","token":"token_a","mode":"resume","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (164 bytes))
{"send_token_info":{"time":"2019-11-15T11:35:50.285+09:00","token":"token_a","mode":"lock","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

#### confirm log messages

`delivery_robot_02` released `token_a`, so `delivery_robot_01` which is wating this token can lock it and can move into restlicted area.

example)

```
2019/11/15 02:35:49 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:35:49 [   INFO] src.token - switch token (token_a) from delivery_robot_02 to delivery_robot_01
2019/11/15 02:35:50 [   INFO] src.api - move robot(delivery_robot_02) to "place_LB" (waypoints=[{'point': {'x': -0.94, 'y': -5.9, 'z': 0}, 'angle': None}, {'point': {'x': 0.7, 'y': -7.4, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -0.15}}]
2019/11/15 02:35:50 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_a, mode=release, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
2019/11/15 02:35:50 [   INFO] src.api - move robot(delivery_robot_01) to "place_L" (waypoints=[{'point': {'x': 0.5, 'y': -2, 'z': 0}, 'angle': None}, {'point': {'x': 4.71, 'y': -1.99, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 02:35:50 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=resume, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
2019/11/15 02:35:50 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=lock, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked, but its owner is switched from `delivery_robot_02` to `delivery_robot_01`, and `delivery_robot_01` is no longer waiting for this token.
* `token_b` and `token_c` is not locked.

### 16. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:15:55.885+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -0.8, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.8}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -0.8, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:40:34.011437+09:00"}
```

### 17. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:15:58.885+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.71, y: -1.99, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 4.71, y: -1.99, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (538 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 4.71, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.99, "x": 4.71, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:41:03.269719+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (97 bytes))
{"send_state":{"time":"2019-11-15T11:41:03.635+09:00","state":"picking","destination":"倉庫1"}}
```

#### confirm log messages

example)
```
2019/11/15 02:41:03 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 02:41:03 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=moving, next_state=picking, destination=倉庫1
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* token_b and token_c is not locked.

### 18. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:15:55.885+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.90, y: -5.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: -0.94, y: -5.9, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (496 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.9, "x": -0.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -5.9, "x": -0.94, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:43:03.972561+09:00"}
```

### 19. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:15:57.885+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
destination:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:43:27.652796+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (201 bytes))
{"send_cmd":{"time":"2019-11-15T11:43:27.822+09:00","cmd":"navi","waypoints":[{"point":{"x":1.7,"y":-8,"z":0},"angle":null},{"point":{"x":2.3,"y":-7.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0.05}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T11:43:27.874+09:00","token":"token_b","mode":"lock","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:43:27.822+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.7
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
---
```

#### confirm log messages

example)

```
2019/11/15 02:43:27 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:43:27 [   INFO] src.token - lock token (token_b) by delivery_robot_02
2019/11/15 02:43:27 [   INFO] src.api - move robot(delivery_robot_02) to "place_RB" (waypoints=[{'point': {'x': 1.7, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 2.3, 'y': -7.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0.05}}]
2019/11/15 02:43:27 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_b, mode=lock, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* now, `token_b` is locked by `delivery_robot_02`.
* `token_c` is not locked yet.

### 20. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:21:42.334+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 2.1, y: -6.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.2}
destination:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.2, "roll": 0.0, "pitch": 0.0}, "point": {"y": -6.1, "x": 2.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:52:03.358945+09:00"}
```

### 21. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:21:45.334+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
destination:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:52:39.742774+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (152 bytes))
{"send_cmd":{"time":"2019-11-15T11:52:39.920+09:00","cmd":"navi","waypoints":[{"point":{"x":4.9,"y":-7.3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T11:52:39.983+09:00","token":"token_b","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_02"}}
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmdexe', ... (256 bytes))
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:52:39.920+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 4.9
      y: -7.3
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
---
```

#### confirm log messages

example)

```
2019/11/15 02:52:39 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:52:39 [   INFO] src.token - release token (token_b) by delivery_robot_02
2019/11/15 02:52:39 [   INFO] src.api - move robot(delivery_robot_02) to "place_LC" (waypoints=[{'point': {'x': 4.9, 'y': -7.3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0}}]
2019/11/15 02:52:40 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_b, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_02
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is released.
* `token_c` is not locked yet.

### 22. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:24:05.383+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 2.4, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (527 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 2.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:55:54.427042+09:00"}
```

### 23. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:24:05.383+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:56:24.566158+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (300 bytes))
{"send_cmd":{"time":"2019-11-15T11:56:24.727+09:00","cmd":"navi","waypoints":[{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":10.6,"y":-8,"z":0},"angle":null},{"point":{"x":10.73,"y":-9.08,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T11:56:24.779+09:00","token":"token_c","mode":"lock","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T11:56:24.727+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 10.73
      y: -9.08
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
2019/11/15 02:56:24 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:56:24 [   INFO] src.token - lock token (token_c) by delivery_robot_02
2019/11/15 02:56:24 [   INFO] src.api - move robot(delivery_robot_02) to "place_Jdash" (waypoints=[{'point': {'x': 6.06, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.56, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 10.6, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 10.73, 'y': -9.08, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 02:56:24 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_c, mode=lock, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is not locked.
* `token_c` is locked by `delivery_robot_02`.

### 24. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:26:02.142+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 6.0, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (528 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 6.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T11:59:04.755947+09:00"}
```

### 25. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:26:03.142+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 10.70, y: -9.01, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.56}
destination:
  point: {x: 10.73 , y: -9.08, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (539 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.56, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.01, "x": 10.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.08, "x": 10.73, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T11:59:28.943973+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (103 bytes))
{"send_state":{"time":"2019-11-15T11:59:29.080+09:00","state":"delivering","destination":"会議室1"}}
```

#### confirm log messages

example)

```
2019/11/15 02:59:29 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 02:59:29 [   INFO] src.api - publish new state to robot ui(robotui_02), current_state=moving, next_state=delivering, destination=会議室1
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`, and no robot is waiting for this token.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 26. emulate the completion of  picking operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 15 Nov 2019 03:00:57 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 94

{"result":"success"}
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (202 bytes))
{"send_cmd":{"time":"2019-11-15T12:00:57.929+09:00","cmd":"navi","waypoints":[{"point":{"x":0.5,"y":-2,"z":0},"angle":null},{"point":{"x":-0.5,"y":-2.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.8}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:00:57.929+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -0.5
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is still locked by `delivery_robot_01`.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 27. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:33:05.593+09:00'
received_time: '2019-11-13T15:33:05.493+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 0.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -0.5
      y: -2.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.8
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (341 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:33:05.493+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:33:05.593+09:00"}}
```

### 28. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:33:05.593+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.6, y: -1.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.01}
destination:
  point: {x: 0.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.9, "x": 0.6, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": 0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:02:11.575926+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (96 bytes))
{"send_state":{"time":"2019-11-15T12:02:11.713+09:00","state":"moving","destination":"倉庫2"}}
```

#### confirm log messages

example)

```
2019/11/15 03:02:11 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
2019/11/15 03:02:11 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=picking, next_state=moving, destination=倉庫2
```

### 29. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:33:08.593+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.5, y: -2.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
destination:
  point: {x: -0.5, y: -2.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.8}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.8, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.5, "x": -0.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:03:19.461786+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (206 bytes))
{"send_cmd":{"time":"2019-11-15T12:03:19.823+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.94,"y":-5.9,"z":0},"angle":null},{"point":{"x":0.7,"y":-7.4,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-0.15}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)
```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T12:03:19.882+09:00","token":"token_a","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_01"}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:03:19.823+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.94
      y: -5.9
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 0.7
      y: -7.4
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -0.15
---
```

#### confirm log messages

example)

```
2019/11/15 03:03:19 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:03:19 [   INFO] src.token - release token (token_a) by delivery_robot_01
2019/11/15 03:03:19 [   INFO] src.api - move robot(delivery_robot_01) to "place_LB" (waypoints=[{'point': {'x': -0.94, 'y': -5.9, 'z': 0}, 'angle': None}, {'point': {'x': 0.7, 'y': -7.4, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -0.15}}]
2019/11/15 03:03:19 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_01
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` has been released.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`.

### 30. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:36:42.767+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.90, y: -5.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: -0.94, y: -5.9, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (496 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.9, "x": -0.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -5.9, "x": -0.94, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:06:32.114819+09:00"}
```

### 31. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:36:45.767+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
destination:
  point: {x: 0.7, y: -7.4, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -0.15}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -0.15, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.4, "x": 0.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:07:21.081359+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (203 bytes))
{"send_cmd":{"time":"2019-11-15T12:07:21.229+09:00","cmd":"navi","waypoints":[{"point":{"x":1.7,"y":-8,"z":0},"angle":null},{"point":{"x":1.71,"y":-9.7,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T12:07:21.272+09:00","token":"token_b","mode":"lock","lock_owner_id":"delivery_robot_01","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:07:21.229+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.7
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 1.71
      y: -9.7
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:07:21 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:07:21 [   INFO] src.token - lock token (token_b) by delivery_robot_01
2019/11/15 03:07:21 [   INFO] src.api - move robot(delivery_robot_01) to "place_G" (waypoints=[{'point': {'x': 1.7, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 1.71, 'y': -9.7, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 03:07:21 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_b, mode=lock, lock_owner_id=delivery_robot_01, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* now, `token_b` has been locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 32. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:38:31.929+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 1.4, y: -5.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 1.7, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -5.1, "x": 1.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 1.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:10:34.546799+09:00"}
```

### 33. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:38:35.929+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 1.71, y: -9.7, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 1.71, y: -9.7, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (536 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.7, "x": 1.71, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.7, "x": 1.71, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:10:58.740764+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (97 bytes))
{"send_state":{"time":"2019-11-15T12:10:58.978+09:00","state":"picking","destination":"倉庫2"}}
```

#### confirm log messages

example)

```
2019/11/15 03:10:58 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:10:59 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=moving, next_state=picking, destination=倉庫2
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 34. emulate the completion of  picking operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 15 Nov 2019 03:11:49 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 104

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (201 bytes))
{"send_cmd":{"time":"2019-11-15T12:11:50.102+09:00","cmd":"navi","waypoints":[{"point":{"x":1.7,"y":-8,"z":0},"angle":null},{"point":{"x":2.3,"y":-7.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0.05}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:11:50.102+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 1.7
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 35. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:41:18.806+09:00'
received_time: '2019-11-13T15:41:18.306+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 1.7
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 2.3
      y: -7.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.05
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (340 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:41:18.306+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -8.0, "x": 1.7, "z": 0.0}}, {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:41:18.806+09:00"}}
```

### 36. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:41:18.906+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 1.4, y: -6.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 1.7, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (493 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -6.1, "x": 1.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 1.7, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:12:56.621923+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (99 bytes))
{"send_state":{"time":"2019-11-15T12:12:56.757+09:00","state":"moving","destination":"会議室2"}}
```

#### confirm log messages

example)

```
2019/11/15 03:12:56 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
2019/11/15 03:12:56 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=picking, next_state=moving, destination=会議室2
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is locked by `delivery_robot_01`.
* `token_c` is locked by `delivery_robot_02`.

### 37. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:41:19.906+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
destination:
  point: {x: 2.3, y: -7.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.05}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.05, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.5, "x": 2.3, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:13:55.012815+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (152 bytes))
{"send_cmd":{"time":"2019-11-15T12:13:55.185+09:00","cmd":"navi","waypoints":[{"point":{"x":4.9,"y":-7.3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":0}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T12:13:55.228+09:00","token":"token_b","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_01"}}
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmdexe', ... (256 bytes))
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:13:55.185+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 4.9
      y: -7.3
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
---
```

#### confirm log messages

example)

```
2019/11/15 03:13:55 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:13:55 [   INFO] src.token - release token (token_b) by delivery_robot_01
2019/11/15 03:13:55 [   INFO] src.api - move robot(delivery_robot_01) to "place_LC" (waypoints=[{'point': {'x': 4.9, 'y': -7.3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 0}}]
2019/11/15 03:13:55 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_b, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_01
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` has been released.
* `token_c` is locked by `delivery_robot_02`.

### 38. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:45:09.945+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 4.1, y: -7.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (527 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.0, "x": 4.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:16:26.917647+09:00"}
```

### 39. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:45:10.945+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 4.9, y: -7.3, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (530 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.3, "x": 4.9, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:16:52.491320+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

`delivery_robot_01` can not lock `token_c`, so `delivery_robot_01` has to take refuge in 'place_U'

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (157 bytes))
{"send_cmd":{"time":"2019-11-15T12:16:52.686+09:00","cmd":"navi","waypoints":[{"point":{"x":5.8,"y":-7.15,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T12:16:52.754+09:00","token":"token_c","mode":"suspend","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:16:52.686+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 5.8
      y: -7.15
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:16:52 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:16:52 [   INFO] src.token - wait token (token_c) by delivery_robot_01
2019/11/15 03:16:52 [   INFO] src.api - take refuge a robot(delivery_robot_01) in "place_U"
2019/11/15 03:16:52 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_c, mode=suspend, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` is not locked.
* `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_02`, and `delivery_robot_01` has been waiting for this token.

### 40. emulate the completion of delivering operation ( `delivery_robot_02`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_02}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 12:33:51 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 121

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (294 bytes))
{"send_cmd":{"time":"2019-11-15T12:20:29.746+09:00","cmd":"navi","waypoints":[{"point":{"x":10.6,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":5,"y":-8.1,"z":0},"angle":{"roll":0,"pitch":0,"yaw":3.14}}]}}
```

#### confirm ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` )

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T12:20:29.746+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 5.0
      y: -8.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
---
```

### 41. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/megarover_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:48:23.940+09:00'
received_time: '2019-11-13T15:48:23.740+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 10.6
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 5.0
      y: -8.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmdexe', ... (461 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:48:23.740+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -8.0, "x": 10.6, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:48:23.940+09:00"}}
```

### 42. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:48:23.940+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 10.1, y: -7.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.5}
destination:
  point: {x: 10.6, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.5, "roll": 0.0, "pitch": 0.0}, "point": {"y": -7.1, "x": 10.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 10.6, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:21:53.907278+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (104 bytes))
{"send_state":{"time":"2019-11-15T12:21:54.100+09:00","state":"moving","destination":"待機場所２"}}
```

#### confirm log messages

example)

```
2019/11/15 03:21:53 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=standby, next_mode=navi
2019/11/15 03:21:54 [   INFO] src.api - publish new state to robot ui(robotui_02), current_state=delivering, next_state=moving, destination=待機場所２
```

### 43. emulate the stop of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:48:25.940+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 5.0, y: -8.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
destination:
  point: {x: 5.0, y: -8.1, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:22:38.714340+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (253 bytes))
{"send_cmd":{"time":"2019-11-15T12:22:38.935+09:00","cmd":"navi","waypoints":[{"point":{"x":0.2,"y":-8.1,"z":0},"angle":null},{"point":{"x":-1.66,"y":-6.68,"z":0},"angle":null},{"point":{"x":-1.5,"y":-3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (167 bytes))
{"send_token_info":{"time":"2019-11-15T12:22:38.983+09:00","token":"token_c","mode":"release","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T12:22:38.935+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.2
      y: -8.1
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.66
      y: -6.68
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.5
      y: -3.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 1.57
---
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (251 bytes))
{"send_cmd":{"time":"2019-11-15T12:22:39.086+09:00","cmd":"navi","waypoints":[{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":9.43,"y":-9.1,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (166 bytes))
{"send_token_info":{"time":"2019-11-15T12:22:39.137+09:00","token":"token_c","mode":"resume","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (164 bytes))
{"send_token_info":{"time":"2019-11-15T12:22:39.186+09:00","token":"token_c","mode":"lock","lock_owner_id":"delivery_robot_01","prev_owner_id":"delivery_robot_02"}}
```

#### confirm that a ros message for `delivery_robot_01` ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:22:39.086+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 9.43
      y: -9.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

`delivery_robot_02` released `token_c`, so `delivery_robot_01` which is wating this token can lock it and can move into restlicted area.

example)

```
2019/11/15 03:22:38 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 03:22:38 [   INFO] src.token - switch token (token_c) from delivery_robot_02 to delivery_robot_01
2019/11/15 03:22:38 [   INFO] src.api - move robot(delivery_robot_02) to "place_LAdash" (waypoints=[{'point': {'x': 0.2, 'y': -8.1, 'z': 0}, 'angle': None}, {'point': {'x': -1.66, 'y': -6.68, 'z': 0}, 'angle': None}, {'point': {'x': -1.5, 'y': -3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 1.57}}]
2019/11/15 03:22:39 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_c, mode=release, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
2019/11/15 03:22:39 [   INFO] src.api - move robot(delivery_robot_01) to "place_Idash" (waypoints=[{'point': {'x': 6.06, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.56, 'y': -8, 'z': 0}, 'angle': None}, {'point': {'x': 9.43, 'y': -9.1, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 03:22:39 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_c, mode=resume, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
2019/11/15 03:22:39 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_c, mode=lock, lock_owner_id=delivery_robot_01, prev_owner_id=delivery_robot_02
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` and `token_b` is not locked.
* `token_c` is still locked, but its owner is switched from `delivery_robot_02` to `delivery_robot_01`, and `delivery_robot_01` is no longer waiting for this token.

### 44. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:51:45.158+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 0.2, y: -8.1, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (492 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.1, "x": 0.2, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:28:13.567042+09:00"}
```

### 45. emulate the stop of  `delivery_robot_02`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:51:48.158+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: -3.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: -1.5, y: -3.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (536 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -3.0, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -3.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:30:24.891028+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (154 bytes))
{"send_cmd":{"time":"2019-11-15T12:29:05.948+09:00","cmd":"navi","waypoints":[{"point":{"x":-1.5,"y":-2,"z":0},"angle":{"roll":0,"pitch":0,"yaw":1.57}}]}}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T12:29:06.002+09:00","token":"token_a","mode":"lock","lock_owner_id":"delivery_robot_02","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T12:29:05.948+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -1.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:29:05 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 03:29:05 [   INFO] src.token - lock token (token_a) by delivery_robot_02
2019/11/15 03:29:06 [   INFO] src.api - move robot(delivery_robot_02) to "place_RAdash" (waypoints=[{'point': {'x': -1.5, 'y': -2, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 1.57}}]
2019/11/15 03:29:06 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_a, mode=lock, lock_owner_id=delivery_robot_02, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` has been locked by `delivery_robot_02` again.
* `token_b` is not locked.
* `token_c` is still locked.

### 46. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:52:45.158+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -1.5, y: -3.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (495 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -3.0, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:38:27.546435+09:00"}
```

### 47. emulate the stop of  `delivery_robot_02`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:51:48.158+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:39:28.865412+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/cmd', ... (156 bytes))
{"send_cmd":{"time":"2019-11-15T12:39:29.043+09:00","cmd":"navi","waypoints":[{"point":{"x":-1.5,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T12:39:29.109+09:00","token":"token_a","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_02"}}
```

#### confirm that a ros message for delivery_robot_02 ( `/robot_bridge/megarover_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_02"
type: "delivery_robot"
time: "2019-11-15T12:39:29.043+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -1.5
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:39:28 [   INFO] src.api - update robot state, robot_id=delivery_robot_02, current_mode=navi, next_mode=standby
2019/11/15 03:39:28 [   INFO] src.token - release token (token_a) by delivery_robot_02
2019/11/15 03:39:29 [   INFO] src.api - move robot(delivery_robot_02) to "place_V" (waypoints=[{'point': {'x': -1.5, 'y': 1.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 03:39:29 [   INFO] src.api - publish new token_info to robot ui(robotui_02), token=token_a, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_02
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` has been released again.
* `token_b` is not locked.
* `token_c` is still locked.

### 48. emulate the move of  `delivery_robot_02`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:53:45.158+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -1.5, y: -1.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (495 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -1.9, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:43:58.633994+09:00"}
```

### 49. emulate the stop of  `delivery_robot_02`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/megarover_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_02'
type: 'delivery_robot'
time: '2019-11-13T15:51:48.158+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: 1.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: 1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_02/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_02/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:44:58.807244+09:00"}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_02/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_02/cmd', ... (105 bytes))
{"send_state":{"time":"2019-11-15T12:44:58.930+09:00","state":"standby","destination":"待機場所２"}}
```

#### confirm the `delivery_robot_02` entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_02}?type=${DELIVERY_ROBOT_TYPE}&attrs=remaining_waypoints_list" | jq .

confirm below:

* `remaining_waypoints_list` is empty list

### 50. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:51:48.158+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.01}
destination:
  point: {x: 6.06, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (495 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.01, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:46:37.516428+09:00"}
```

### 51. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:51:49.458+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 9.43, y: -9.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: 9.43, y: -9.1, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (536 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.1, "x": 9.43, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -9.1, "x": 9.43, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:47:09.836192+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (100 bytes))
{"send_state":{"time":"2019-11-15T12:47:09.991+09:00","state":"picking","destination":"会議室2"}}
```

#### confirm log messages

example)

```
2019/11/15 03:47:09 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:47:10 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=moving, next_state=picking, destination=会議室2
```

### 52. emulate the completion of delivering operation ( `delivery_robot_01`)

#### post a movenext data

In [ ]:
TOKEN=$(cat ~/core/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens | map(select(.allowed_paths[] | contains ("^/controller/.*$"))) | .[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Content-Type: application/json" https://api.tech-sketch.jp/controller/api/v1/robots/${DELIVERY_ROBOT_01}/nexts/ -X PATCH -d '{}'

example)

```
HTTP/1.1 200 OK
server: envoy
date: Fri, 08 Nov 2019 13:18:24 GMT
content-type: application/json
content-length: 21
access-control-allow-origin: *
x-envoy-upstream-service-time: 124

{"result":"success"}
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (247 bytes))
{"send_cmd":{"time":"2019-11-15T12:48:08.711+09:00","cmd":"navi","waypoints":[{"point":{"x":9.56,"y":-8,"z":0},"angle":null},{"point":{"x":6.06,"y":-8,"z":0},"angle":null},{"point":{"x":5,"y":-8.1,"z":0},"angle":{"roll":0,"pitch":0,"yaw":3.14}}]}}
```

#### confirm ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` )

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:48:08.711+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 5.0
      y: -8.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
---
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` and `token_b` is not locked.
* `token_c` is still locked by `delivery_robot_01`.

### 53. notify that the command is received

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.
```
rostopic pub -1 /robot_bridge/turtlebot_01/cmdexe uoa_poc2_msgs/r_result "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:59:08.798+09:00'
received_time: '2019-11-13T15:59:07.798+09:00'
received_cmd: 'navi'
received_waypoints:
  -
    point:
      x: 9.56
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 6.06
      y: -8.0
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: 5.0
      y: -8.1
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 3.14
result: 'ack'
errors: ['']
"
```

#### confirm the MQTT message

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmdexe', ... (401 bytes))
{"send_cmd": {"received_time": "2019-11-13T15:59:07.798+09:00", "errors": [""], "received_cmd": "navi", "received_waypoints": [{"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, {"angle": null, "point": {"y": -8.0, "x": 6.06, "z": 0.0}}, {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}], "result": "ack", "time": "2019-11-13T15:59:08.798+09:00"}}
```

### 54. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:59:08.798+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 9.56, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.5}
destination:
  point: {x: 9.56, y: -8.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (494 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.5, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.0, "x": 9.56, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:49:59.145086+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (102 bytes))
{"send_state":{"time":"2019-11-15T12:49:59.312+09:00","state":"moving","destination":"待機場所1"}}
```

#### confirm log messages

example)

```
2019/11/15 03:49:59 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=standby, next_mode=navi
2019/11/15 03:49:59 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=picking, next_state=moving, destination=待機場所1
```

### 55. emulate the stop of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T15:59:09.798+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: 5.0, y: -8.1, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
destination:
  point: {x: 5.0, y: -8.1, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 3.14}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (532 bytes))
{"errors": [], "pose": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 3.14, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.1, "x": 5.0, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:51:21.937192+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (253 bytes))
{"send_cmd":{"time":"2019-11-15T12:51:22.116+09:00","cmd":"navi","waypoints":[{"point":{"x":0.2,"y":-8.1,"z":0},"angle":null},{"point":{"x":-1.66,"y":-6.68,"z":0},"angle":null},{"point":{"x":-1.5,"y":-3,"z":0},"angle":{"roll":0,"pitch":0,"yaw":1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T12:51:22.164+09:00","token":"token_c","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_01"}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:51:22.116+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: 0.2
      y: -8.1
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.66
      y: -6.68
      z: 0.0
    angle_optional:
      valid: False
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 0.0
  -
    point:
      x: -1.5
      y: -3.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:51:22 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:51:22 [   INFO] src.token - release token (token_c) by delivery_robot_01
2019/11/15 03:51:22 [   INFO] src.api - move robot(delivery_robot_01) to "place_LAdash" (waypoints=[{'point': {'x': 0.2, 'y': -8.1, 'z': 0}, 'angle': None}, {'point': {'x': -1.66, 'y': -6.68, 'z': 0}, 'angle': None}, {'point': {'x': -1.5, 'y': -3, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 1.57}}]
2019/11/15 03:51:22 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_c, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_01
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* all token is not locked.

### 56. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:02:08.895+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: 0.1, y: -8.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
destination:
  point: {x: 0.2, y: -8.1, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 0.0}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (492 bytes))
{"errors": [], "pose": {"angle": {"yaw": 0.0, "roll": 0.0, "pitch": 0.0}, "point": {"y": -8.0, "x": 0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -8.1, "x": 0.2, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T12:54:11.144649+09:00"}
```

### 57. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:02:09.895+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: -3.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -3.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T12:56:45.051384+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (154 bytes))
{"send_cmd":{"time":"2019-11-15T12:56:45.220+09:00","cmd":"navi","waypoints":[{"point":{"x":-1.5,"y":-2,"z":0},"angle":{"roll":0,"pitch":0,"yaw":1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (147 bytes))
{"send_token_info":{"time":"2019-11-15T12:56:45.272+09:00","token":"token_a","mode":"lock","lock_owner_id":"delivery_robot_01","prev_owner_id":""}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T12:56:45.220+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -1.5
      y: -2.0
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: 1.57
---
```

#### confirm log messages

example)

```
2019/11/15 03:56:45 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 03:56:45 [   INFO] src.token - lock token (token_a) by delivery_robot_01
2019/11/15 03:56:45 [   INFO] src.api - move robot(delivery_robot_01) to "place_RAdash" (waypoints=[{'point': {'x': -1.5, 'y': -2, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': 1.57}}]
2019/11/15 03:56:45 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=lock, lock_owner_id=delivery_robot_01, prev_owner_id=
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* `token_a` has been locked by `delivery_robot_01` again.
* `token_b` is not locked.
* `token_c` is still locked.

### 58. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:03:08.895+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -1.5, y: -2.9, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: false
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (495 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.9, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": null, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T13:05:32.320707+09:00"}
```

### 59. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:04:09.895+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
destination:
  point: {x: -1.5, y: -2.0, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: 1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": 1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": -2.0, "x": -1.5, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T13:06:11.663138+09:00"}
```

#### confirm that a MQTT message to `/delivery_robot/delivery_robot_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/cmd', ... (156 bytes))
{"send_cmd":{"time":"2019-11-15T13:06:11.829+09:00","cmd":"navi","waypoints":[{"point":{"x":-0.4,"y":1.5,"z":0},"angle":{"roll":0,"pitch":0,"yaw":-1.57}}]}}
```

#### confirm that a `lock` MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (150 bytes))
{"send_token_info":{"time":"2019-11-15T13:06:11.879+09:00","token":"token_a","mode":"release","lock_owner_id":"","prev_owner_id":"delivery_robot_01"}}
```

#### confirm that a ros message for delivery_robot_01 ( `/robot_bridge/turtlebot_01/cmd` ) is sent automatically

example)

```yaml
id: "delivery_robot_01"
type: "delivery_robot"
time: "2019-11-15T13:06:11.829+09:00"
cmd: "navi"
waypoints:
  -
    point:
      x: -0.4
      y: 1.5
      z: 0.0
    angle_optional:
      valid: True
      angle:
        roll: 0.0
        pitch: 0.0
        yaw: -1.57
---
```

#### confirm log messages

example)

```
2019/11/15 04:06:11 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 04:06:11 [   INFO] src.token - release token (token_a) by delivery_robot_01
2019/11/15 04:06:11 [   INFO] src.api - move robot(delivery_robot_01) to "place_Q" (waypoints=[{'point': {'x': -0.4, 'y': 1.5, 'z': 0}, 'angle': {'roll': 0, 'pitch': 0, 'yaw': -1.57}}]
2019/11/15 04:06:11 [   INFO] src.api - publish new token_info to robot ui(robotui_01), token=token_a, mode=release, lock_owner_id=, prev_owner_id=delivery_robot_01
```

#### confirm token entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${TOKEN_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/entities/?type=${TOKEN_TYPE} | jq .

confirm below:

* all token is not locked.

### 60. emulate the move of  `delivery_robot_01`

#### send a ros message 

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:05:08.895+09:00'
mode: 'navi'
errors: ['']
pose:
  point: {x: -0.1, y: 1.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: -0.4, y: 1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (531 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.1, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "navi", "time": "2019-11-15T13:09:57.762869+09:00"}
```

### 61. emulate the stop of  `delivery_robot_01`

#### send a ros message

_On ROS Kinect PC_
1. start a terminal
1. execute below command.

```
rostopic pub -1 /robot_bridge/turtlebot_01/state uoa_poc2_msgs/r_state "
id: 'delivery_robot_01'
type: 'delivery_robot'
time: '2019-11-13T16:07:09.895+09:00'
mode: 'standby'
errors: ['']
pose:
  point: {x: -0.4, y: 1.5, z: 0.0}
  angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
destination:
  point: {x: -0.4, y: 1.5, z: 0.0}
  angle_optional:
    valid: true
    angle: {roll: 0.0, pitch: 0.0, yaw: -1.57}
covariance: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
  0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
battery:
  voltage: 0.0
  current_optional: {valid: false, current: 0.0}
"
```

#### confirm the MQTT message to `/delivery_robot/delivery_robot_01/attrs`

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/delivery_robot/delivery_robot_01/attrs', ... (534 bytes))
{"errors": [], "pose": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "battery": {"current": null, "voltage": 0.0}, "destination": {"angle": {"yaw": -1.57, "roll": 0.0, "pitch": 0.0}, "point": {"y": 1.5, "x": -0.4, "z": 0.0}}, "covariance": [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], "mode": "standby", "time": "2019-11-15T13:10:41.200048+09:00"}
```

#### confirm that a MQTT message to `/robot_ui/robotui_01/cmd` is sent automatically

example)

```
Client mosq-TrZhDaCwOfCzPWvm9w received PUBLISH (d0, q0, r0, m0, '/robot_ui/robotui_01/cmd', ... (103 bytes))
{"send_state":{"time":"2019-11-15T13:10:41.381+09:00","state":"standby","destination":"待機場所1"}}
```

#### confirm log messages

example)

```
2019/11/15 04:10:41 [   INFO] src.api - update robot state, robot_id=delivery_robot_01, current_mode=navi, next_mode=standby
2019/11/15 04:10:41 [   INFO] src.api - publish new state to robot ui(robotui_01), current_state=moving, next_state=standby, destination=待機場所1
```

#### confirm the `delivery_robot_02` entity

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-Servicepath: ${DELIVERY_ROBOT_SERVICEPATH}" "https://api.${DOMAIN}/orion/v2/entities/${DELIVERY_ROBOT_01}?type=${DELIVERY_ROBOT_TYPE}&attrs=remaining_waypoints_list" | jq .

confirm below:

* `remaining_waypoints_list` is empty list